In [15]:
# include modules relative to this file's directory
include(joinpath(@__DIR__, "../src/utils.jl"))
include(joinpath(@__DIR__, "../src/model.jl"))
include(joinpath(@__DIR__, "../src/analysis.jl"))

using .Utils
using .Model
using .Analysis
using Plots, LaTeXStrings

# make Figures directory if missing
isdir(joinpath(@__DIR__, "..", "figures")) || mkpath(joinpath(@__DIR__, "..", "figures"))

# ==== parameters ====
k = 1.0
K = 15.0
ξ = 0.1
ωn = 8.0
θ_ref = 0.5
dt = 1e-3
c = 0.2

# search ranges
β_range = range(0.01, stop=0.5, length=500)
w_range = range(0.1, stop=12, length=500)

# find best (w, beta) using Analysis
w_best, β_best, _ = Analysis.find_best_beta_w(β_range, w_range; K=K, ωn=ωn, ξ=ξ, k=k, θ_ref=θ_ref)
println("Found w_best=", w_best, " β_best=", β_best)

# compute γ_ref / γ_opt
β_ref = β_best
ω_ref = w_best

# damped natural frequency
ωd = ωn * sqrt(1 - ξ^2)
γ_opt = β_ref * (1 - exp(-c * 2 * π / ωd)) / (3 * exp(-c * π / (2 * ωd)) - exp(-c * 3 * π / (2 * ωd)))

c_arr = [i for i = 0.01: 0.01 :1^1]
γ_arr = [i for i = 0.01: 0.01 :1^1]
log_c = 10 .^(range(-2, stop=0, length=length(c_arr)))
log_γ = 10 .^(range(-3, stop=-1, length=length(c_arr)))
# log_γ = range(0.001, stop=0.2, length=length(c_arr))

A_anal = zeros(length(c_arr), length(γ_arr))
for i in 1:length(log_c)
    c = log_c[i]
    for j in 1:length(log_γ)
        γ = log_γ[j] 
        
        c2 = γ * exp(-c*(pi/(2*ωd)))
        c1 = exp(-c*(pi/(ωd)))
        c0 = (1 - c1)*β_ref
        
        if c2 == c0
            β_est = β_ref
            Aij = min(abs(Analysis.func_th_beta(β_est; K=K, ωn=ωn, ωd=ωd, ξ=ξ, k=k) -  θ_ref), 0.5)
        end
        if c2 < c0
            β_est = c2/(1 - c1)
            Aij = min(abs(Analysis.func_th_beta(β_est; K=K, ωn=ωn, ωd=ωd, ξ=ξ, k=k) -  θ_ref), 0.5)
        end
        if c2 > c0
            β_est_u = c2 - c0 + β_ref
            Aij = min(abs(Analysis.func_th_beta(β_est_u; K=K, ωn=ωn, ωd=ωd, ξ=ξ, k=k) -  θ_ref), 0.5)
        end
        A_anal[i, j] = Aij
    end
end

# plots
h = heatmap(range(-2, stop=0, length=length(c_arr)),
    range(-3, stop=-1, length=length(c_arr)), A_anal, 
    xlabel = "γ", ylabel = "c",
    xlabelfontsize=8,  
    ylabelfontsize=8,
    tickfontsize=7,
    # size=(504, 288)
    size=(504, 184)
)

# save the figure into project-level /figures
outpath = joinpath(@__DIR__, "..", "figures", "fig3_2.pdf")
savefig(h, outpath)
println("Saved ", outpath)


Found w_best=7.707414829659319

 β_best=0.09150300601202405
Saved /Users/taya/Documents/neuromorphicControl/pendulum/simulation/fig-repro/scripts/../figures/fig3_2.pdf
